In [ ]:
col_labels = ['Lab', 'publication', 'Experiment Type', 'Biosample', 'Dataset', 'Num Experiment Sets', 'Year relases', 'Link to Browse Page', 'Status of release']

In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import pandas as pd

my_key = get_key('default')
dekker_grant = ff_utils.search_metadata('/search/?type=ExperimentSetReplicate&award.uuid=ae6c618f-7a8c-441e-a886-e30bbbe591da', key = my_key)

In [ ]:
#### Investigative Phase ############

In [ ]:
sample_item = dekker_grant[2]
print(sample_item.keys())

In [ ]:
item_publication = sample_item['publications_of_set'][0]['display_title']

In [ ]:
print(item_publication)

In [ ]:
title = item_publication[:item_publication.index(')') + 1]

In [ ]:
print(title)

In [ ]:
word = ''
for l in item_publication:
    word = word + l
    if l == ')':
        break
    

In [ ]:
print(word)

In [ ]:
for key in sample_item:
    item_lab = sample_item['lab']['display_title']
    item_publication = sample_item['publications_of_set'] # if publication, then publication 'title'
    item_experiment_in_set = sample_item['experiments_in_set']
    item_biosource = item_experiment_in_set[0]['biosample']['biosource'][0]['cell_line']['display_title']
    item_status = sample_item['status']
    item_project_released = sample_item['project_release']
    item_exp_type = item_experiment_in_set[0]['experiment_type']

In [ ]:
print(item_lab)

In [ ]:
print(item_publication)

In [ ]:
#print(item_experiment_in_set)

In [ ]:
for exp in item_experiment_in_set:
    exp_sample_biosample = exp['biosample']['biosource'][0]['cell_line']['display_title']
    exp_sample_type = exp['experiment_type']
    break

In [ ]:
print(item_biosource)

In [ ]:
print(exp_sample_type)

In [ ]:
print(item_exp_type)

In [ ]:
print(item_status)

In [ ]:
print(item_project_released)

In [ ]:
###################################

In [ ]:
import collections
my_dict = collections.defaultdict(dict)

In [ ]:
datasets = []
print(len(dekker_grant))
for exp_replicate_set in dekker_grant:
    exp_replicate_set_keys = exp_replicate_set.keys()
    lab = exp_replicate_set['lab']['display_title']
    publication = exp_replicate_set['publications_of_set']
    if publication:
        publication_title = [publication[x]['display_title'][:-len(publication[x]['title'])] for x in range(len(publication))]
        if len(publication_title) == 1:
            publication_title = publication_title[0]
            
    else:
        publication_title = ''
    experiment_in_set = exp_replicate_set['experiments_in_set']
    biosource = experiment_in_set[0]['biosample']['biosource'][0]['cell_line']['display_title']
    status = exp_replicate_set['status']
    project_release_date = exp_replicate_set['project_release'] if 'project_release' in exp_replicate_set_keys else 'none'
    exp_type = experiment_in_set[0]['experiment_type']
    dataset = exp_replicate_set['dataset_label'] if 'dataset_label' in exp_replicate_set_keys else exp_type + ' on ' + biosource 
    datasets.append(dataset)
    public_release_date = exp_replicate_set['public_release'] if 'public_release' in exp_replicate_set_keys else ''
    if dataset not in my_dict.keys():
        my_dict[dataset]= collections.defaultdict(dict)

    my_dict[dataset][exp_replicate_set['uuid']] = {'lab':lab, 'publication': publication_title, 'experiment_type': exp_type, 'biosample':biosource, 'dataset': dataset, 'public_release_date':public_release_date, 'project_release_date': project_release_date, 'status': status}
    
    

In [ ]:
print(len(my_dict["in situ Hi-C on H1-ESC cells"].keys()))

In [ ]:
#print(datasets)

In [ ]:
#print(my_dict)

In [ ]:
# for key,item in my_dict['DNA FISH on Chr18'].items():
#     print(key, item)
#     print()

In [ ]:
for dataset_key, dataset in my_dict.items():
    dataset_label = collections.defaultdict(dict)
    x = 0
    notes_list = []
    for uuid, exp_set in dataset.items():
        diff_list = []
        if x == 0:
            dataset_label['lab'] = exp_set['lab']
            dataset_label['publication'] =  exp_set['publication']
            dataset_label['experiment_type'] = exp_set['experiment_type']
            dataset_label['biosample'] = exp_set['biosample']
            dataset_label['dataset'] = exp_set['dataset']
            dataset_label['status'] = exp_set['status']
            dataset_label['public_release_date'] = exp_set['public_release_date']
            dataset_label['project_release_date'] = exp_set['project_release_date']
            x = x + 1
            continue
        
        for key in dataset_label:
            if exp_set[key] != dataset_label[key]:
                if key == 'project_release_date' or key == 'public_release_date':
                    if dataset_label[key] < exp_set[key]:
                        dataset_label[key] = exp_set[key]
                else:
                    print(key)
                    print(exp_set[key],dataset_label[key])
                    diff_list.append(dataset_label[key])
                    diff_list.append(exp_set[key])
                    data_label[key]= set(diff_list)
    dataset_label['num_exp_set'] = len(dataset.keys())
    my_dict[dataset_key]['out_info'] = dataset_label

In [ ]:
# for key,item in my_dict["in situ Hi-C on H1-ESC cells"].items():
#     print(key, item)
#     print()

In [ ]:
for key,item in my_dict['DNA FISH on Chr18'].items():
    pub = item['publication']


In [ ]:
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)

workbook_id = '1v4J-2a5e6a3mbZAI2B3Yk2z1WFdQI5ZNktlXCbx_MOM'
workbook = gc.open_by_key(workbook_id)

In [ ]:
print(pub[0])

In [ ]:
worksheet = workbook.add_worksheet('Data Plan Table', 100, 20)

In [ ]:
gs_write = []
row = 1
col = 0
for dataset in my_dict.values():
    row = row + 1
    for key in dataset['out_info'].values():
        col = col + 1
        gs_write.append(gspread.models.Cell(row,col,key))
    col = 0    
    

In [ ]:
print(gs_write)

In [ ]:
worksheet.update_cells(gs_write)

In [ ]:
col_labels = ['Lab', 'publication', 'Experiment Type', 'Biosample', 'Dataset', 'Status of release', 'Public release date', 'Project release date', 'Num Experiment Sets']

In [ ]:
gs_write = []
row = 1
col = 1
for item in col_labels:
    gs_write.append(gspread.models.Cell(row,col,item))
    col = col + 1

In [ ]:
worksheet.update_cells(gs_write)

In [ ]:
############################# SCRIPT TO RUN ################################################


In [1]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import pandas as pd

my_key = get_key('default')
all_grants = ff_utils.search_metadata('/search/?type=Award', key = my_key)

In [2]:
for grant in all_grants:
    #print(grant)
    break

In [3]:
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)

google_management_sheet_id = '1Mk5SH8JftJNB1zdE8bUfAAA23GV6wkp-UUkIboSba4A'
google_management_spreadsheet = gc.open_by_key(google_management_sheet_id)
google_management_sheet = google_management_spreadsheet.sheet1
#print(google_management_sheet)
grant_sheets_links = google_management_sheet.get_all_values()
for item in grant_sheets_links:
    spreadsheet_id = item[2]

In [4]:
grants = []
for item in grant_sheets_links:
    if 'Guttman' in item[0]:
        guttman_grants = item[1].split(',')
        for award in guttman_grants:
            for grant in all_grants:
                if award in grant['link_id']:
                    grants.append(grant)
    else:
        for grant in all_grants:
            if item[1] in grant['link_id']:
                grants.append(grant)          

In [5]:
print(len(grants))

37


In [ ]:
# for grant in all_grants:
#     print(grant['link_id'])

In [6]:
for item in all_grants:
    if item['uuid'] == '2850ac0b-92f4-4c8b-8f6a-5199afc442b1':
        grants.append(item)

In [7]:
print(len(grants))

38


In [ ]:
#grants[38]

In [8]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import pandas as pd
import collections
import time
from gspread_formatting import *

my_key = get_key('default')

In [10]:
ct = 0
for grant in grants:   
    if grant['uuid'] != 'fcc7f634-9252-499f-b79c-380795af2ddd':
        continue
    if grant['uuid'] == "8a9d6db4-37eb-488e-828c-d90f3ab348f6":
        continue
    grant_uuid = grant['uuid']
    q ='/search/?type=ExperimentSetReplicate&award.uuid={}'.format(grant_uuid)
    exps_in_grant = ff_utils.search_metadata(q, key = my_key)
    ct = ct + 1
    if len(exps_in_grant) == 0:
        
        for item in grant_sheets_links:
            if 'Guttman' in item[0]:
                guttman_grants = item[0].split(',')
                for gut_grant in guttman_grants:
                    if gut_grant in grant['link_id']:
                        grant_gs_id = item[2]
            else:         
                if item[1] in grant['link_id']:
                    grant_gs_id = item[2]
        
        header_message = 'There have been no data submissions from your center yet. Please add rows below for expected submission through August 2020. Please see link below for more examples.'
        link_template = '=hyperlink("https://docs.google.com/spreadsheets/d/1BbB2p20wrnlc1isiGYp5_-F6BmHCHNDREm7k_CzNQrc/edit#gid=0","examples")'
        tail_message1 = ['my lab', 'hi-c variation project', 'in situ Hi-C', 'U2OS', 'hi-c protocol variatons on h1', '5', 'planned', '2019-12-20']
        tail_message2 = ['collaborator lab', 'ctcf tracking project', 'SPT', 'U2OS', 'CTCF tracking on U2OS', '30', 'planned', '2020-03-31']      
        s3 = boto3.resource('s3')
        obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
        cont = obj.get()['Body'].read().decode()
        key_dict = json.loads(cont)
        SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
        creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
        gc = gspread.authorize(creds)

        workbook_id = grant_gs_id 
        workbook = gc.open_by_key(workbook_id)

        print('working on', workbook)

        try:
            worksheet = workbook.add_worksheet('Data Status Plans', 100, 20)

        except:
            worksheet = workbook.worksheet('Data Status Plans')
  
        gs_write = []
        gs_write.append(gspread.models.Cell(1,1,header_message))
        gs_write.append(gspread.models.Cell(2,1, link_template))
        col = 1
        col_labels = ['Lab', 'Publication/Project', 'Experiment Type', 'Biosample', 'Dataset','Num Experiment Sets', 'Status of release', 'Public release date', 'Project release date', 'Browser Link']
        for i in col_labels:
            gs_write.append(gspread.models.Cell(3,col,i))
            col = col + 1
        col = 0
        for txt in tail_message1:
            col = col + 1
            gs_write.append(gspread.models.Cell(4, col, txt))
        col = 0
        for txt in tail_message2:
            col = col + 1
            gs_write.append(gspread.models.Cell(5, col, txt))
        
        print(gs_write) 
        worksheet.update_cells(gs_write,value_input_option='USER_ENTERED')
        fmt1 = cellFormat(textFormat=textFormat(foregroundColor=color(0.98,0.01,0.01))) #red color
        fmt2 = cellFormat(textFormat=textFormat(bold=True)) # black bold
        format_cell_ranges(worksheet, [('A4:J5', fmt1)])
        format_cell_ranges(worksheet, [('A1:J3', fmt2)])
        
        try:
            worksheet.update_title('Data Status/Plans')
        except:
            print('Already exists')
            
        if ct == 5:
            time.sleep(100)
            ct = 0
        
        
    else:
        header_message = 'Below is the list of data submitted by your center so far. Please add rows below for expected submission through August 2020. Please see link below for more examples.'
        link_template = '=hyperlink("https://docs.google.com/spreadsheets/d/1BbB2p20wrnlc1isiGYp5_-F6BmHCHNDREm7k_CzNQrc/edit#gid=0","examples")'
        tail_message1 = ['my lab', 'hi-c variation project', 'in situ Hi-C', 'U2OS', 'hi-c protocol variatons on h1', '5', 'planned', '2019-12-20']
        tail_message2 = ['collaborator lab', 'ctcf tracking project', 'SPT', 'U2OS', 'CTCF tracking on U2OS', '30', 'planned', '2020-03-31']      
                     
                       
        print(grant['link_id'])
        print(len(exps_in_grant))

        my_dict = {}
        my_dict = collections.defaultdict(dict)
        for exp_replicate_set in exps_in_grant:
            exp_replicate_set_keys = exp_replicate_set.keys()
            lab = exp_replicate_set['lab']['display_title']
            publication = exp_replicate_set['publications_of_set']

            if publication:
                publication_title = [publication[x]['display_title'][:publication[x]['display_title'].index(")") + 1] for x in range(len(publication))]
                if len(publication_title) == 1:
                    publication_title = publication_title[0]

            else:
                publication_title = ''

            experiment_in_set = exp_replicate_set['experiments_in_set']
            biosource = experiment_in_set[0]['biosample']['biosource_summary']
            status = exp_replicate_set['status']
            project_release_date = exp_replicate_set['project_release'] if 'project_release' in exp_replicate_set_keys else ''
            exp_type = experiment_in_set[0]['experiment_type']

            dataset_title_iskey = False
            if 'dataset_label' in exp_replicate_set_keys:
                dataset_title = exp_replicate_set['dataset_label'] 
                dataset_key = dataset_title + '/' + exp_type
                dataset_title_iskey = True
            else:
                dataset_title = exp_type + ' on ' + biosource 
                dataset_key = dataset_title
            public_release_date = exp_replicate_set['public_release'] if 'public_release' in exp_replicate_set_keys else ''

            if dataset_key not in my_dict.keys():
                my_dict[dataset_key] = collections.defaultdict(dict)
            
            my_dict[dataset_key][exp_replicate_set['uuid']] = {'lab':lab, 'publication': publication_title, 'experiment_type': exp_type, 'biosample':biosource,'dataset_title': dataset_title, 'public_release_date':public_release_date, 'project_release_date': project_release_date, 'status': status, 'dataset_title_iskey':dataset_title_iskey}
 
        for dataset_key, dataset in my_dict.items():
            dataset_label = collections.defaultdict(dict)

            x = 0
            num_biosample = 0
            num_status = 0
            for uuid, exp_set in dataset.items():
                if x == 0:
                    dataset_label['lab'] = exp_set['lab']
                    dataset_label['publication'] =  exp_set['publication']
                    dataset_label['experiment_type'] = exp_set['experiment_type']
                    dataset_label['biosample'] = exp_set['biosample']
                    dataset_label['dataset_title'] = exp_set['dataset_title']
                    dataset_label['num_exp_set'] = len(dataset.keys())
                    dataset_label['status'] = exp_set['status']
                    dataset_label['public_release_date'] = exp_set['public_release_date']
                    dataset_label['project_release_date'] = exp_set['project_release_date']
                    dataset_label['dataset_title_iskey'] = exp_set['dataset_title_iskey']
                    x = x + 1
                    continue
                for key in dataset_label:
                    if key == 'num_exp_set':
                        continue
                    if exp_set[key] != dataset_label[key]:
                        if key == 'project_release_date' or key == 'public_release_date':
                            if dataset_label[key] < exp_set[key]:
                                dataset_label[key] = exp_set[key]
                                continue
                        elif key == 'biosample':
                            num_biosample = num_biosample + 1
                        elif key == 'status':
                            num_status = num_status + 1                                           
                        else:
                            print(key, dataset_key)
                            print(exp_set[key],dataset_label[key])



            if dataset_label['dataset_title_iskey']:
                browser_link = 'https://data.4dnucleome.org/browse/?award.uuid={}&dataset_label={}&experiments_in_set.experiment_type={}'.format(grant_uuid, dataset_label['dataset_title'], dataset_label['experiment_type'])
                browser_link_fotmat = '=HYPERLINK("{}", "View in Portal")'.format(browser_link)
            else:
                browser_link = "https://data.4dnucleome.org/browse/?award.uuid={}&dataset_label=No+value&experiments_in_set.biosample.biosource_summary={}&experiments_in_set.experiment_type={}".format(grant_uuid, dataset_label['biosample'], dataset_label['experiment_type'])
                browser_link_fotmat = '=HYPERLINK("{}", "View in Portal")'.format(browser_link)
                dataset_label['dataset_title'] = ''

            dataset_label.pop('dataset_title_iskey')
            dataset_label['browser_view'] = browser_link_fotmat
            
            if num_biosample > 0:
                dataset_label['biosample'] = dataset_label['biosample'] + " and {} other(s)".format(num_biosample)
            
            if num_status > 0:
                dataset_label['status'] = 'Partially released'
            
            my_dict[dataset_key]['out_info'] = dataset_label   

        list_datasets = []
        for k in my_dict:
            interest = my_dict[k]["out_info"]
            temp = {}
            temp['dataset_label'] = k
            for k_sub, v_sub in interest.items():
                temp[k_sub] = v_sub
            list_datasets.append(temp)
        list_datasets = sorted(list_datasets, key=lambda x: (x['publication'],x['lab'], x['experiment_type'] ))    

        for item in grant_sheets_links:
            if 'Guttman' in item[0]:
                guttman_grants = item[1].split(',')
                for gut_grant in guttman_grants:
                    if gut_grant in grant['link_id']:
                        grant_gs_id = item[2]
            if item[1] in grant['link_id']:
                grant_gs_id = item[2]

        s3 = boto3.resource('s3')
        obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
        cont = obj.get()['Body'].read().decode()
        key_dict = json.loads(cont)
        SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
        creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
        gc = gspread.authorize(creds)

        workbook_id = grant_gs_id 
        workbook = gc.open_by_key(workbook_id)

        print('working on', workbook)

        try:
            worksheet = workbook.add_worksheet('Data Status Plans', 100, 20)

        except:
            worksheet = workbook.worksheet('Data Status Plans')


        gs_write = []
        gs_write.append(gspread.models.Cell(1,1,header_message))
        gs_write.append(gspread.models.Cell(2,1, link_template))
        row = 3
        col = 1
        col_labels = ['Lab', 'Publication/Project', 'Experiment Type', 'Biosample', 'Dataset','Num Experiment Sets', 'Status of release', 'Public release date', 'Project release date', 'Browser Link']
        for i in col_labels:
            gs_write.append(gspread.models.Cell(row,col,i))
            col = col + 1
        col = 0
        for dataset in list_datasets:
            row = row + 1
            dataset.pop('dataset_label')
            for key in dataset:
                col = col + 1
                gs_write.append(gspread.models.Cell(row,col,dataset[key]))
            col = 0 
        col = 1
        for txt in tail_message1:
            gs_write.append(gspread.models.Cell(row + 1, col, txt))
            col = col + 1
        col = 1
        for txt in tail_message2:
            gs_write.append(gspread.models.Cell(row + 2, col, txt))
            col = col + 1
                       
              
        worksheet.update_cells(gs_write,value_input_option='USER_ENTERED')
        fmt1 = cellFormat(textFormat=textFormat(foregroundColor=color(0.98,0.01,0.01)))
        format_cell_ranges(worksheet, [('A{}:J{}'.format(row+1, row+2), fmt1)])
        fmt2 = cellFormat(textFormat=textFormat(bold=True))
        format_cell_ranges(worksheet, [('A1:J3', fmt2)])
        
        
        
#         for x in range(0,7):
#             if x == 0:
#                 try:
#                     worksheet = workbook.worksheet('Data Plan Table')
#                 except:
#                     continue
#             else:
#                 try:
#                     worksheet = workbook.worksheet('Data Plan Table {}'.format(x))
#                 except:
#                     continue
#             workbook.del_worksheet(worksheet)
        try:
            worksheet.update_title('Data Status/Plans')
        except:
            print('Already exists')
            
        if ct == 10:
            time.sleep(100)
            ct = 0
        print()

~awards~1U54DK107979-01~
59


TypeError: can only concatenate str (not "dict") to str

In [14]:
exp_type

{'title': 'RNA-seq',
 'display_title': 'RNA-seq',
 'principals_allowed': {'edit': ['group.admin'],
  'view': ['system.Everyone'],
  'audit': ['system.Everyone']},
 '@id': '/experiment-types/rna-seq/',
 'uuid': '19f848e6-7c52-4b8b-8aa8-e82ea0a968ce',
 'link_id': '~experiment-types~rna-seq~'}

In [ ]:
gs_write 

In [ ]:
dir(workbook)

In [ ]:
browser_link_dataset = 'https://data.4dnucleome.org/browse/?dataset_label={}'.format(dataset)



In [ ]:
browser_link_no_dataset"https://data.4dnucleome.org/browse/?award.center_title=NOFIC+-+Dekker&dataset_label=No+value&experiments_in_set.biosample.biosource_summary=animal+zygote&experiments_in_set.experiment_type=single+cell+Hi-C&experimentset_type=replicate&type=ExperimentSetReplicate".format(biosource_summary)




In [ ]:
browser_link_no_dataset = 'https://data.4dnucleome.org/browse/?award.uuid={}&dataset_label={}&experiments_in_set.biosample.biosource_summary={}&experiments_in_set.experiment_type={}'.format(grant_uuid, dataset_label, biosource, exp_type)


In [ ]:
print(browser_link_no_dataset)

In [ ]:
browser_link_dataset = 'https://data.4dnucleome.org/browse/?award.uuid={}&dataset_label={}'.format(grant_uuid, dataset_label)


In [ ]:
my_dict = {'dataset1':{'title': 'A', 'Lab':'F', 'exp': 'T'}, 'dataset2':{'title': 'B', 'Lab':'C', 'exp': 'W'}, 'dataset3':{'title': 'a', 'Lab':'D', 'exp': 'W'}}

In [ ]:
my_dict

In [ ]:
sorted_dict = sorted()
    

In [ ]:
order = 0
orderd_dict = []
for dataset in my_dict:
    first = dataset
    
    

In [ ]:
print(item)

In [ ]:
print(type(item[0]))

In [ ]:
for item in grants:
    print(type(item['lab']))
    break

In [ ]:
browser_link = "https://data.4dnucleome.org/browse/?award.uuid={}&dataset_label=No+value&experiments_in_set.biosample.biosource_summary={}&experiments_in_set.experiment_type={}".format(grant_uuid, dataset_label['biosample'], dataset_label['experiment_type'])
browser_link_fotmat = 'HYPERLINK("{}", "View in Portal")'.format(browser_link)



In [ ]:
print(browser_link)

In [ ]:
print(browser_link_fotmat)

In [ ]:
print(type(browser_link_fotmat))

In [ ]:
'=' + browser_link_fotmat

In [ ]:
workbook

In [ ]:
dir(workbook)

In [ ]:
print(workbook.worksheets())

In [ ]:
workbook.del_worksheet('Data Plan Table 6')

In [ ]:
help(workbook.del_worksheet)

In [ ]:
worksheet_list = workbook.worksheets()

In [ ]:
print(worksheet_list)

In [ ]:
worksheet = workbook.worksheet("Data Plan Table 3")

In [ ]:
workbook.del_worksheet(worksheet)

In [ ]:
dir(gspread.models.Cell)

In [ ]:
worksheet

In [ ]:
workbook

In [ ]:
worksheet.update_acell('S2', formula)

In [ ]:
formula = worksheet.acell('J2', value_render_option = 'FORMULA').value


In [ ]:
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)

workbook_id = "1v4J-2a5e6a3mbZAI2B3Yk2z1WFdQI5ZNktlXCbx_MOM"
workbook = gc.open_by_key(workbook_id)

In [ ]:
workbook

In [ ]:
worksheet = workbook.worksheet('Data Status Plans')

In [ ]:
worksheet

In [ ]:
data1 = worksheet.acell('J2',value_render_option = 'UNFORMATTED_VALUE' )

In [ ]:
print(data1)

In [ ]:
type(data1)

In [ ]:
worksheet.update_acell('M3', browser_link_fotmat)
                       

In [ ]:
browser_link_fotmat

In [ ]:
cell_lists = [3,13,browser_link_fotmat,3,14,browser_link_fotmat]

In [ ]:
print(cell_lists)

In [ ]:
worksheet.update_cells(cell_lists)

In [ ]:
dir(worksheet)

In [ ]:
help(worksheet.update_cell)

In [ ]:
help(worksheet.update_acell)

In [ ]:
help(worksheet.update_cells)

In [ ]:
cell_list = worksheet.range('J2:J19')

In [ ]:
for cell in cell_list:
    formatted = '=HYPERLINK("{}", "View in Portal")'.format(cell.value)
    print(cell)
    cell.value = formatted

In [ ]:
worksheet.update_cells(cell_list)

In [ ]:
worksheet.values_update('N3', browser_link_fotmat)

In [ ]:
for cell in cell_list:
    #formatted = '=HYPERLINK("{}", "View in Portal")'.format(cell.value)
    #cell.value = formatted
    print(cell)

In [ ]:
for cell in cell_list:
    #formatted = '=HYPERLINK("{}", "View in Portal")'.format(cell.value)
    #print(cell)
    cell.value = formatted

In [ ]:
worksheet.update_cells(cell_list, value_input_option='USER_ENTERED')

In [ ]:
workbook

In [ ]:
help(worksheet.update_cells)

In [ ]:
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)

workbook_id = "1v4J-2a5e6a3mbZAI2B3Yk2z1WFdQI5ZNktlXCbx_MOM"
workbook = gc.open_by_key(workbook_id)

In [ ]:
workbook

In [ ]:
worksheet = workbook.worksheet('Data Status Plans')

In [ ]:
worksheet

In [ ]:
tail_message_1 = worksheet.range('A22:H22')
tail_message_2 = worksheet.range('A23:H23')                                

In [ ]:
print(tail_message_1 )

In [ ]:
from gspread_formatting import *

In [ ]:
fmt = cellFormat(textFormat=textFormat(bold=True, foregroundColor=color(0,0,0))) #black

In [ ]:
fmt2 = cellFormat(textFormat=textFormat(bold=True, foregroundColor=color(0.98,0.01,0.01))) #color red and bold font

In [ ]:
worksheet

In [ ]:
workbook

In [ ]:
format_cell_ranges(worksheet, [('A1', fmt)])

In [ ]:
help(format_cell_ranges)

In [ ]:
format_cell_ranges(worksheet, [('A3:J3', fmt)])

In [ ]:
workbook

In [ ]:
worksheet.update_title('Data Status/Plans')

In [ ]:
dir(worksheet)